In [1]:
import json
import time
import os
from pytket.passes import FullPeepholeOptimise, KAKDecomposition
from pytket import Circuit, Qubit
from pytket.circuit import PauliExpBox, OpType, fresh_symbol
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit.display import render_circuit_jupyter
from pytket.passes import DecomposeBoxes, PauliSimp, DecomposeSingleQubitsTK1
from pytket.extensions.qiskit import tk_to_qiskit

In [2]:
# Pytket
pauli_dict={"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}
def pytket_run(parr):
    print(parr)
    all_symbols=[fresh_symbol(f"s{x}") for x in range(len(parr))]
    n = len(parr[0][0])
    q = [Qubit(i) for i in range(n)]
    def to_pauli_list(ps):
        r = []
        for i in ps:
            r.append(pauli_dict[i])
        return r
    # for i in parr:
    all_qubit_pauli_strings=[[QubitPauliString(q,to_pauli_list(p)) for p in block] for block in parr]

    def add_excitation(circ, all_ops, param=1.0):
        for term, symbol in zip(all_ops, all_symbols):
            # each term is one block so they should have the same parameter symbol.
            for pauli_str in term:
                qubits, paulis = zip(*pauli_str.to_list())
                paulis=[pauli_dict[p] for p in paulis]
                pbox = PauliExpBox(paulis, symbol * param)
                # transform the string form of the qubits to qubits
                qubits=[circ.get_q_register(q_list[0])[q_list[1][0]] for q_list in qubits]
                circ.add_pauliexpbox(pbox, qubits)
    ansatz = Circuit(n)
    # print(type(ansatz))
    add_excitation(ansatz, all_qubit_pauli_strings)
    # render_circuit_jupyter(ansatz)
    PauliSimp().apply(ansatz)
    # DecomposeBoxes().apply(ansatz)
    FullPeepholeOptimise().apply(ansatz) #heavy optimization
    DecomposeSingleQubitsTK1().apply(ansatz)
    # KAKDecomposition().apply(ansatz)
    # render_circuit_jupyter(ansatz)
    print(f"CNOT: {ansatz.n_gates_of_type(OpType.CX)}, Single: {ansatz.n_gates-ansatz.n_gates_of_type(OpType.CX)}, Total: {ansatz.n_gates}, Depth: {ansatz.depth()}")

    return ansatz

def is_all_identity(pauli):
    return all(char == 'I' for char in pauli)


# def update_json_file(filename, runtime, cx_count, circ_depth):
#     with open(f'benchmarks/results/test_' + filename, 'r') as paulis_file:
#         data=json.load(paulis_file)
#     print(data[1])
#     # with open(f'benchmarks/results/test_' + filename, 'w') as paulis_file:
#     #     json.dump([test_paulis, results], paulis_file, indent=4)

In [3]:
#Compare json files in a specific folder
def run_experiment_folder(folder_path = None, filename = None, save_output = False):

    if filename == None:
        file_list = os.listdir(folder_path)
    else:
        file_list = [filename]
    # Iterate over all files in the folder
    for filename in file_list:
        # Check if the file is a JSON file
        if filename.endswith(".json"):
            results = []
            # Print the filename
            print(filename)
            with open(folder_path + '/' + filename, "r") as file:
                test_paulis = json.load(file)
            # Filter the list to remove all identity Paulis
            if filename.startswith(("max_cut", "labs")): # qaoa requires a block structure
                paulis=[[p for p in block if not is_all_identity(p)] for block in test_paulis]
                paulis=[block for block in paulis if len(block)!=0]
                number_of_ham=len([p for block in paulis for p in block])
            else: # not qaoa so not block structure
                paulis = [[p] for p in test_paulis if not is_all_identity(p)]
                number_of_ham=len(paulis)

            # Measure time for Tetris method
            start_time = time.time()
            circuit = pytket_run(paulis)
            end_time = time.time()
            circuit_qiskit=tk_to_qiskit(circuit)
            # circuit_qiskit = transpile(circuit_qiskit, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
            tot_time = end_time - start_time
        
            # Collect results
            result = {
                "num_paulis": number_of_ham,
                "times": {
                    "pytket_time": tot_time
                },
                "gate_counts": {
                    "pytket_method": circuit.n_gates_of_type(OpType.CX)
                },
                "circuit_entangling_depth": {
                    "pytket_method": circuit_qiskit.depth(lambda instr: len(instr.qubits) > 1)
                },
                "test_paulis_file": f'benchmarks/results/test_pytket_' + filename
            }
            print(result)
            results.append(result)
            if save_output == True:
                # Save test_paulis to a separate JSON file
                with open(f'benchmarks/results/test_pytket_' + filename, 'w') as paulis_file:
                    json.dump([test_paulis, results], paulis_file, indent=4)
    

    

In [4]:
#Compare a given list of paulis
def run_experiment_paulis(test_paulis, test_params = None, save_output = False, filename = "Paulis"):

    results = []
    # Filter the list to remove all identity Paulis
    if test_params is None:
        test_params = [0.01 * i for i in range(len(test_paulis))]
    
    number_of_ham=len([p for block in test_paulis for p in block])
    # Measure time for Tetris method
    start_time = time.time()
    circuit = pytket_run(test_paulis)
    end_time = time.time()
    circuit_qiskit=tk_to_qiskit(circuit)
    # circuit_qiskit = transpile(circuit_qiskit, basis_gates=["cx", "sx", "x", "rz"], optimization_level=3)
    tot_time = end_time - start_time

    # Collect results
    result = {
        "num_paulis": number_of_ham,
        "times": {
            "pytket_time": tot_time
        },
        "gate_counts": {
            "pytket_method": circuit.n_gates_of_type(OpType.CX)
        },
        "circuit_entangling_depth": {
            "pytket_method": circuit_qiskit.depth(lambda instr: len(instr.qubits) > 1)
        },
        "test_paulis_file": f'benchmarks/results/test_pytket_' + filename
    }
    print(result)
    results.append(result)
    if save_output == True:
        # Save test_paulis to a separate JSON file
        with open(f'benchmarks/results/test_pytket_' + filename, 'w') as paulis_file:
            json.dump([test_paulis, results], paulis_file, indent=4)
    return circuit #sorted_entanglers_f
    

    

In [9]:
#first compare the UCCSD ansatz
electrons_list = [2, 2, 4, 6, 8, 10,]# 12]
orbitals_list = [4, 6, 8, 12, 16, 20]#, 24]
#First evaluate the UCCSD ansatz:
for e, o in zip(electrons_list, orbitals_list):
    filename=f"uccsd_hamiltonian_e{e}_o{o}.json"
    with open(f"benchmarks/uccsd_hamiltonians/{filename}", "r") as file:
        data=json.load(file)
    # test_paulis = generate_UCCSD_entanglers(x, y)
    print(data)
    data = [[p for p in block if not is_all_identity(p)] for block in data]
    data= [block for block in data if len(block)!=0]
    num_hams=len([p for block in data for p in block])
    print(data)
    entanglers = run_experiment_paulis(data, save_output = True, filename=f"Paulis{num_hams}.json")

[['YZXI', 'XZXI', 'YZYI', 'XZYI'], ['IYZX', 'IXZX', 'IYZY', 'IXZY'], ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']]
[['YZXI', 'XZXI', 'YZYI', 'XZYI'], ['IYZX', 'IXZX', 'IYZY', 'IXZY'], ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']]
[['YZXI', 'XZXI', 'YZYI', 'XZYI'], ['IYZX', 'IXZX', 'IYZY', 'IXZY'], ['YXYX', 'XXYX', 'YYYX', 'XYYX', 'YXYY', 'XXYY', 'YYYY', 'XYYY', 'YXXX', 'XXXX', 'YYXX', 'XYXX', 'YXXY', 'XXXY', 'YYXY', 'XYXY']]
CNOT: 53, Single: 67, Total: 120, Depth: 83
{'num_paulis': 24, 'times': {'pytket_time': 2.219089984893799}, 'gate_counts': {'pytket_method': 53}, 'circuit_entangling_depth': {'pytket_method': 50}, 'test_paulis_file': 'benchmarks/results/test_pytket_Paulis24.json'}
[['YZXIII', 'XZXIII', 'YZYIII', 'XZYIII'], ['YZZZXI', 'XZZZXI', 'YZZZYI', 'XZZZYI'], ['IYZXII', 'IXZXII', 'IYZYII', 'IXZYII'], ['IYZZZX', 

In [10]:
#Then compare the Hamiltonian simulation paulis in HS_paulis folder
run_experiment_folder(folder_path = "benchmarks/HS_paulis", save_output = True)

benzene.json
[['IIIIIIIIIIIZ'], ['IIIIIIIIIIYY'], ['IIIIIIIIIIXX'], ['IIIIIIIIIYZY'], ['IIIIIIIIIXZX'], ['IIIIIIIIYZZY'], ['IIIIIIIIXZZX'], ['IIIIIIIYZZZY'], ['IIIIIIIXZZZX'], ['IIIIIIYZZZZY'], ['IIIIIIXZZZZX'], ['IIIIIIIIIIZI'], ['IIIIIIIIIYYI'], ['IIIIIIIIIXXI'], ['IIIIIIIIYZYI'], ['IIIIIIIIXZXI'], ['IIIIIIIYZZYI'], ['IIIIIIIXZZXI'], ['IIIIIIYZZZYI'], ['IIIIIIXZZZXI'], ['IIIIIIIIIZII'], ['IIIIIIIIYYII'], ['IIIIIIIIXXII'], ['IIIIIIIYZYII'], ['IIIIIIIXZXII'], ['IIIIIIYZZYII'], ['IIIIIIXZZXII'], ['IIIIIIIIZIII'], ['IIIIIIIYYIII'], ['IIIIIIIXXIII'], ['IIIIIIYZYIII'], ['IIIIIIXZXIII'], ['IIIIIIIZIIII'], ['IIIIIIYYIIII'], ['IIIIIIXXIIII'], ['IIIIIIZIIIII'], ['IIIIIZIIIIII'], ['IIIIYYIIIIII'], ['IIIIXXIIIIII'], ['IIIYZYIIIIII'], ['IIIXZXIIIIII'], ['IIYZZYIIIIII'], ['IIXZZXIIIIII'], ['IYZZZYIIIIII'], ['IXZZZXIIIIII'], ['YZZZZYIIIIII'], ['XZZZZXIIIIII'], ['IIIIZIIIIIII'], ['IIIYYIIIIIII'], ['IIIXXIIIIIII'], ['IIYZYIIIIIII'], ['IIXZXIIIIIII'], ['IYZZYIIIIIII'], ['IXZZXIIIIIII'], ['YZZZYIIIIIII

In [7]:
# #Then compare the MAXCUT paulis in max_cut_paulis folder

# run_experiment_folder(folder_path = "benchmarks/max_cut_paulis_blocks", save_output = True)
all_filenames=["max_cut_benchmark_regular_n20_e4_l1.json", "max_cut_benchmark_regular_n20_e4_l3.json", 
               "max_cut_benchmark_regular_n20_e8_l1.json", "max_cut_benchmark_regular_n20_e8_l3.json",
               "max_cut_benchmark_regular_n20_e12_l1.json", "max_cut_benchmark_regular_n20_e12_l3.json"]
for filename in all_filenames:
    run_experiment_folder(folder_path = "benchmarks/max_cut_paulis_blocks", filename=filename, save_output = True)

max_cut_benchmark_regular_n20_e4_l1.json
[['IIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIZIIIIIIIIIZ', 'IIIIIIIZIIIIIIIIIIIZ', 'IIIIIIIIIIZIIIIIIIIZ', 'IIIIIIIIIIIZIIIIIIZI', 'ZIIIIIIIIIIIIIIIIIZI', 'IIIIIIZIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIIIZZII', 'IIIIIZIIIIIIIIIIIZII', 'IIIZIIIIIIIIIIIIIZII', 'IZIIIIIIIIIIIIIIIZII', 'IIIIIIIIIIIIZIIIZIII', 'IZIIIIIIIIIIIIIIZIII', 'IIIIIIIZIIIIIIIIZIII', 'IIIIIIIIIIIIIIZZIIII', 'IIIIIIIIIIIIIZIZIIII', 'IIIIIIIIIZIIIIIZIIII', 'IIIZIIIIIIIIIIIZIIII', 'IIIIIIIIIIIIZIZIIIII', 'IIIIIIIIIIIZIIZIIIII', 'IIIIIIIZIIIIIIZIIIII', 'IIIIIIZIIIIIIZIIIIII', 'IIIIIIIIZIIIIZIIIIII', 'IIIIIIIIIIZIIZIIIIII', 'ZIIIIIIIIIIIZIIIIIII', 'IIIZIIIIIIIIZIIIIIII', 'IIZIIIIIIIIZIIIIIIII', 'ZIIIIIIIIIIZIIIIIIII', 'IIIIIIIIIZZIIIIIIIII', 'IIIIIIIIZIZIIIIIIIII', 'IIIIZIIIIZIIIIIIIIII', 'IIIIIIIZZIIIIIIIIIII', 'IZIIIIIIZIIIIIIIIIII', 'IIZIIIZIIIIIIIIIIIII', 'IIIIZIZIIIIIIIIIIIII', 'IIIZIZIIIIIIIIIIIIII', 'IZIIIZIIIIIIIIIIIIII', 'IIZIIZIIIIIIIIIIIIII', 'IIZIZIIIIIIIIIIIIIII', 'ZIIIZIIIIIIIIIIIIIII

In [8]:
# #Labs

run_experiment_folder(folder_path = "benchmarks/labs_paulis_blocks", save_output = True)

labs_n10_layers1.json
[['IIZZZZIIII', 'ZZIIIIZZII', 'ZZIIZZIIII', 'IIZZIIZZII', 'ZZIIIIIIZZ', 'IIZZIZZIII', 'IIZZIIIIZZ', 'ZZIIIZZIII', 'ZZZZIIIIII', 'IIIIZZZZII', 'ZZIIIIIZZI', 'IIIZZZZIII', 'IIZZIIIZZI', 'IZIIIIIIIZ', 'IZIIIZIIII', 'ZIIIIIZIII', 'ZIIIZIIIII', 'IIIIIZIIIZ', 'ZIIIZZIIIZ', 'IZIIZIZIIZ', 'IIIIZIZIII', 'IIIIZZIZZI', 'ZIZIIIIIII', 'IZIZIIIIII', 'IIIIZZIIZZ', 'IZZIIIZZII', 'IIIIIIZZZZ', 'ZZIZZIIIII', 'IZZIIIIIZZ', 'IIIZZIZZII', 'IZZZZIIIII', 'IIIIIZZIZZ', 'IZZIZZIIII', 'IIIIIIIZIZ', 'IIIIIIZIZI', 'IZZIIZZIII', 'IIIZZIIIZZ', 'IZZIIIIZZI', 'IIIZIIIIIZ', 'ZIIZIIZIIZ', 'IIIZIZIIII', 'IIIZZIIZZI', 'IIZIIZZIIZ', 'IIIIIZZZZI', 'IIZIZIIIII', 'IZIZIIZIZI', 'ZIZIIZIZII', 'ZIZIIIZIZI', 'IIIZIIIZII', 'IZIIIIIZII', 'IIIZIZIZIZ', 'IZIZIIIZIZ', 'IZIZIZIZII', 'IIIIIZIZII', 'ZIZIZIZIII', 'IZIZZIZIII', 'ZIZIIIIZIZ', 'IIIIZIZZIZ', 'ZIZZIZIIII', 'IIZIZZIZII', 'ZIIIIIIIZI', 'IIZIZIIZIZ', 'IIIZIZZIZI', 'IIZIZIZIZI', 'ZIIZIZIIZI', 'IIZIIIZIII', 'IZIIZZIIZI', 'ZIIZZIIZII', 'IIZIIIIIZI', 'IIIIZIIIZ